In [4]:
!pip install psycopg2-binary
!pip install delta-spark==2.0.2

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [5]:
import psycopg2
import numpy as np

import os
from datetime import datetime

import findspark
findspark.init('/home/nasty280700/spark-3.2.4-bin-hadoop3.2')
#findspark.init('/home/nasty280700/.local/lib/python3.10/site-packages/pyspark')
import pyspark
from pyspark.sql import SparkSession

from pyspark.sql.utils import AnalysisException
from pyspark.sql.functions import lit
from pyspark.sql.functions import when, coalesce, col
from delta.tables import DeltaTable

In [6]:
def create_mirrow_with_table (table_name:str, delta_dir:str, primary_keys:list):
    print('подключение к базе данных')
    
    conn = psycopg2.connect(user='postgres',
                            password = '',
                            host='127.0.0.1',
                            port='5432',
                            database='postgres')
    cursor = conn.cursor()
    
    get_table_col_name= f"SELECT column_name FROM information_schema.columns WHERE table_name = '{table_name}' ORDER BY ordinal_position"
    cursor.execute(get_table_col_name)
    col_name = cursor.fetchall()
    
    query = f"SELECT * FROM {table_name}"
    cursor.execute(query)
    df_data = cursor.fetchall()
    
    create_table_query = """
    CREATE TABLE IF NOT EXISTS public.log_1 (
        operation_start_time timestamp,
        delta_id varchar(5),
        start_or_end varchar(5),
        table_name varchar (15)
    )
    """
    conn.commit()
    cursor.execute(create_table_query)

    print('создание spark сессии')
    
    spark = (SparkSession.builder
             .config("spark.memory.offHeap.enabled","true")
             .config("spark.memory.offHeap.size","16g")
             .appName('project_2.2')
             .enableHiveSupport()
             .getOrCreate())
    
    spark_df = spark.createDataFrame(df_data, list(zip(*col_name))[0])
    spark_df = spark_df.withColumn('account_rk', col('account_rk').cast('int'))
    spark_df = spark_df.withColumn('currency_rk', col('currency_rk').cast('int'))
    spark_df.show(3)
    
    delta_ids = []
    for folder_name in sorted(os.listdir(delta_dir)):
        if len(folder_name) == 4:
            delta_ids.append(folder_name)
    get_checked_deltas = """
        SELECT delta_id FROM public.log_1 WHERE start_or_end = 'END'
    """
    cursor.execute(get_checked_deltas)
    checked_deltas = cursor.fetchall()
    conn.commit()
    
    if len(checked_deltas) == 0:
        pass
    else:
        checked_deltas = list(zip(*checked_deltas))[0]
        
    for id, del_id in enumerate(sorted(set(delta_ids).symmetric_difference(set(checked_deltas)))):
        print('считывается дельта', del_id)
        
        current_time = datetime.now()
        insert_start_to_log = f"""
            INSERT INTO public.log_1
            VALUES ('{current_time}', '{del_id}', 'START', '{table_name}');
            """
        cursor.execute(insert_start_to_log)
        conn.commit()
        
        if del_id[-1] == '0':
            path = delta_dir + del_id + '/md_account_d' + '.csv'
        else:
            path = delta_dir + del_id + '/md_account_d' + '_' + del_id[-1] + '.csv'
        delta_df = spark.read.csv(path, header=True, sep = ';')
        delta_df = delta_df.na.drop("all")
    
        for col_name in delta_df.columns:
            delta_df = delta_df.withColumnRenamed(col_name, col_name.lower())
            
        try:
            delta_data = delta_df.exceptAll(spark_df)
        except AnalysisException:
            for new_columns in (sorted(set(delta_df.columns).symmetric_difference(set(spark_df.columns)))):
                spark_df = spark_df.withColumn(new_columns, lit(None))
        delta_data = delta_df.exceptAll(spark_df)

        for col_name in spark_df.columns:
            spark_df = spark_df.withColumnRenamed(col_name, 'spr_' + col_name)
    
        condition = None
        for key in primary_keys:
            if condition is None:
                condition = col(f'{key}') == col(f'spr_{key}')
                when_condition_del = col(f'{key}').isNull()
            else:
                condition = condition & (col(f'{key}') == col(f'spr_{key}'))
                when_condition_del = when_condition_del & (col(f'{key}').isNull()) 
                
        join_data = delta_data.join(spark_df, condition, how='outer')
    
        join_data = join_data.withColumn('delete_flag', when(when_condition_del, 'y')
                                          .otherwise('n'))

        mirrow = join_data.select('delete_flag',
        *[when(col('delete_flag') == 'n', col(column)).otherwise(col('spr_' + column)).alias(column)
        for column in delta_data.columns])
        
        #update_del = join_data.filter(col('delete_flag') == 'y').select(*spark_df.columns, 'delete_flag')
        #update_del.show(5)
        #for i, col_name in enumerate(update_del.columns):
         #   if i != len(update_del.columns)-1:
          #      update_del = update_del.withColumnRenamed(col_name, delta_data.columns[i])    
        #mirrow = update_del.union(join_data.filter(col('delete_flag') == 'n').select(*delta_data.columns, 'delete_flag'))
        print('количество записей в зеркале', mirrow.count())
        mirrow.toPandas().to_csv('mirr_md_account_d/md_account.csv', index = False)
        mirrow.show(5)
        mirrow = mirrow.drop('delete_flag')
        spark_df = mirrow
        
        current_time = datetime.now()
        insert_end_to_log = f"""
            INSERT INTO public.log_1 
            VALUES ('{current_time}', '{del_id}', 'END', '{table_name}');
            """
        cursor.execute(insert_end_to_log)
        conn.commit()
    spark.stop()
    cursor.close()
    conn.commit()
    conn.close()

In [8]:
create_mirrow_with_table('md_account_d', 'data_deltas/', ['account_rk'])

подключение к базе данных
создание spark сессии


+----------------+--------------------+----------+--------------------+---------+-----------+-------------+
|data_actual_date|data_actual_end_date|account_rk|      account_number|char_type|currency_rk|currency_code|
+----------------+--------------------+----------+--------------------+---------+-----------+-------------+
|      2018-01-01|          2018-01-31|  36237725|30425840700000583001|        A|         35|          840|
|      2018-01-01|          2018-01-31|     24656|30114840700000770002|        A|         35|          840|
|      2018-01-01|          2018-01-31|  18849846|30109810500000435003|        P|         34|          643|
+----------------+--------------------+----------+--------------------+---------+-----------+-------------+
only showing top 3 rows

считывается дельта 1004


количество записей в зеркале 112


+-----------+----------------+--------------------+----------+--------------------+---------+-----------+-------------+---------+---------+----------------+
|delete_flag|data_actual_date|data_actual_end_date|account_rk|      account_number|char_type|currency_rk|currency_code|client_id|branch_id|open_in_internet|
+-----------+----------------+--------------------+----------+--------------------+---------+-----------+-------------+---------+---------+----------------+
|          y|      2018-01-01|          2018-01-31|     13560|30110810300000008001|        A|         34|          643|     null|     null|            null|
|          y|      2018-01-01|          2018-01-31|     13597|30110840900000008002|        A|         35|          840|     null|     null|            null|
|          y|      2018-01-01|          2018-01-31|     13598|30114840100000009001|        A|         35|          840|     null|     null|            null|
|          y|      2018-01-01|          2018-01-31|     13

In [5]:
create_mirrow_with_table('md_account_d', 'data_deltas/', ['account_rk'])

подключение к базе данных
создание spark сессии


+----------------+--------------------+----------+--------------------+---------+-----------+-------------+
|data_actual_date|data_actual_end_date|account_rk|      account_number|char_type|currency_rk|currency_code|
+----------------+--------------------+----------+--------------------+---------+-----------+-------------+
|      2018-01-01|          2018-01-31|  36237725|30425840700000583001|        A|         35|          840|
|      2018-01-01|          2018-01-31|     24656|30114840700000770002|        A|         35|          840|
|      2018-01-01|          2018-01-31|  18849846|30109810500000435003|        P|         34|          643|
+----------------+--------------------+----------+--------------------+---------+-----------+-------------+
only showing top 3 rows

считывается дельта 1004


количество удалённых строк 110


количество обновлённых строк 2


количество вставленных строк 0
количество записей в зеркале 112


In [5]:
from delta import *

builder = pyspark.sql.SparkSession.builder.appName("MyApp") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")

spark = configure_spark_with_delta_pip(builder).getOrCreate()
df = spark.read.csv('data_deltas/1000/md_account_d.csv', sep = ';', header = True)
df.write.format("delta").save('data_deltas/1000.csv')
df = spark.read.format("delta").load("data_deltas/1000.csv")

23/09/17 23:10:17 WARN SparkSession: Cannot use io.delta.sql.DeltaSparkSessionExtension to configure session extensions.
java.lang.ClassNotFoundException: io.delta.sql.DeltaSparkSessionExtension
	at java.base/java.net.URLClassLoader.findClass(URLClassLoader.java:476)
	at java.base/java.lang.ClassLoader.loadClass(ClassLoader.java:594)
	at java.base/java.lang.ClassLoader.loadClass(ClassLoader.java:527)
	at java.base/java.lang.Class.forName0(Native Method)
	at java.base/java.lang.Class.forName(Class.java:398)
	at org.apache.spark.util.Utils$.classForName(Utils.scala:216)
	at org.apache.spark.sql.SparkSession$.$anonfun$applyExtensions$1(SparkSession.scala:1194)
	at org.apache.spark.sql.SparkSession$.$anonfun$applyExtensions$1$adapted(SparkSession.scala:1192)
	at scala.collection.mutable.ResizableArray.foreach(ResizableArray.scala:62)
	at scala.collection.mutable.ResizableArray.foreach$(ResizableArray.scala:55)
	at scala.collection.mutable.ArrayBuffer.foreach(ArrayBuffer.scala:49)
	at org.a

Py4JJavaError: An error occurred while calling o500.csv.
: org.apache.spark.SparkException: Cannot find catalog plugin class for catalog 'spark_catalog': org.apache.spark.sql.delta.catalog.DeltaCatalog
	at org.apache.spark.sql.errors.QueryExecutionErrors$.catalogPluginClassNotFoundForCatalogError(QueryExecutionErrors.scala:1455)
	at org.apache.spark.sql.connector.catalog.Catalogs$.load(Catalogs.scala:65)
	at org.apache.spark.sql.connector.catalog.CatalogManager.loadV2SessionCatalog(CatalogManager.scala:66)
	at org.apache.spark.sql.connector.catalog.CatalogManager.$anonfun$v2SessionCatalog$2(CatalogManager.scala:85)
	at scala.collection.mutable.HashMap.getOrElseUpdate(HashMap.scala:86)
	at org.apache.spark.sql.connector.catalog.CatalogManager.$anonfun$v2SessionCatalog$1(CatalogManager.scala:85)
	at scala.Option.map(Option.scala:230)
	at org.apache.spark.sql.connector.catalog.CatalogManager.v2SessionCatalog(CatalogManager.scala:84)
	at org.apache.spark.sql.connector.catalog.CatalogManage

In [24]:

from delta.pip_utils import configure_spark_with_delta_pip
def create_mirrow_with_deltas (table_name:str, delta_dir:str, primary_keys:list):
    print('подключение к базе данных')
    
    conn = psycopg2.connect(user='postgres',
                            password = '',
                            host='127.0.0.1',
                            port='5432',
                            database='postgres')
    cursor = conn.cursor()
    
    get_table_col_name= f"SELECT column_name FROM information_schema.columns WHERE table_name = '{table_name}' ORDER BY ordinal_position"
    cursor.execute(get_table_col_name)
    col_name = cursor.fetchall()
    
    query = f"SELECT * FROM {table_name}"
    cursor.execute(query)
    
    df_data = cursor.fetchall()
    
    create_table_query = """
    CREATE TABLE IF NOT EXISTS public.log_1 (
        operation_start_time timestamp,
        delta_id varchar(5),
        start_or_end varchar(5),
        table_name varchar (15)
    )
    """
    conn.commit()
    
    cursor.execute(create_table_query)

    print('создание spark сессии')

    
    spark = SparkSession.builder \
        .config("spark.memory.offHeap.enabled","true") \
        .config("spark.memory.offHeap.size","16g") \
        .config("spark.jars.packages", "io.delta:delta-core_2.12:1.2.1") \
        .config("spark.jars.repositories", "https://maven-central.storage-download.googleapis.com/maven2/") \
        .appName('project_2.2') \
        .enableHiveSupport() \
        .getOrCreate()
    spark_df = spark.createDataFrame(df_data, list(zip(*col_name))[0])
    spark_df.show(3)
    

    delta_ids = []
    for folder_name in sorted(os.listdir(delta_dir)):
        if len(folder_name) == 4:
            delta_ids.append(folder_name)
            
    get_checked_deltas = """
        SELECT delta_id FROM public.log_1 WHERE start_or_end = 'END'
    """
    cursor.execute(get_checked_deltas)
    checked_deltas = cursor.fetchall()
    conn.commit()
    if len(checked_deltas) == 0:
        pass
    else:
        checked_deltas = list(zip(*checked_deltas))[0]
        
    for id, del_id in enumerate(sorted(set(delta_ids).symmetric_difference(set(checked_deltas)))):
        print('считывается дельта', del_id)
        
        current_time = datetime.now()
        insert_start_to_log = f"""
            INSERT INTO public.log_1
            VALUES ('{current_time}', '{del_id}', 'START', '{table_name}');
            """
        cursor.execute(insert_start_to_log)
        conn.commit()
        if del_id[-1] == '0':
            path = delta_dir + del_id + '/md_account_d' + '.csv'
        else:
            path = delta_dir + del_id + '/md_account_d' + '_' + del_id[-1] + '.csv'
        delta_df = spark.read.csv(path, header=True, sep = ';')
        delta_df = delta_df.na.drop("all")
        #delta_table = DeltaTable.forPath(spark, path)
        #delta_df = spark.read.format("delta").load(path)
        #delta_df = (spark_builder.read.format("delta").load(path))

        for col_name in delta_df.columns:
            delta_df = delta_df.withColumnRenamed(col_name, col_name.lower())
        try:
            delta_data = spark_df.exceptAll(delta_df)
        except AnalysisException:
            for new_columns in (set(delta_df.columns).symmetric_difference(set(spark_df.columns))):
                spark_df = spark_df.withColumn(new_columns, lit(None))
        delta_data = spark_df.exceptAll(delta_df)
    
        for col_name in delta_data.columns:
            delta_data = delta_data.withColumnRenamed(col_name, 'del_' + col_name)
    
        condition = None
        for key in primary_keys:
            if condition is None:
                condition = col(f'{key}') == col(f'del_{key}')
                when_condition_1 = col(f'del_{key}').isNull()
                when_condition_2 = col(f'{key}').isNull()
            else:
                condition = condition & (col(f'{key}') == col(f'del_{key}'))
                when_condition_1 = when_condition_1 & (col(f'del_{key}').isNull()) 
                when_condition_2 = when_condition_2 & (col(f'{key}').isNull())  
        
        delta_data = delta_data.to_pandas_on_spark()
        delta_data.to_delta(mode='overwrite', path= '1000.csv')
        delta_data = spark.read.format("delta").load("1000.csv")
        
        (delta_data
        .merge(spark_df, condition)
        .whenMatchedUpdateAll(
            {
                
            })
        .whenNotMatchedInsertAll()
        .execute()
        )
        
        current_time = datetime.now()
        #insert_end_to_log = f"""
         #   INSERT INTO public.log_1 
          #  VALUES ('{current_time}', '{del_id}', 'END', '{table_name}');
           # """
        #cursor.execute(insert_end_to_log)
        #conn.commit()
    spark.stop()
    cursor.close()
    conn.commit()
    conn.close()

In [25]:
create_mirrow_with_deltas('md_account_d', 'data_deltas/', ['account_rk'])

подключение к базе данных
создание spark сессии
+----------------+--------------------+--------------------+--------------------+---------+--------------------+-------------+
|data_actual_date|data_actual_end_date|          account_rk|      account_number|char_type|         currency_rk|currency_code|
+----------------+--------------------+--------------------+--------------------+---------+--------------------+-------------+
|      2018-01-01|          2018-01-31|36237725.00000000...|30425840700000583001|        A|35.00000000000000...|          840|
|      2018-01-01|          2018-01-31|24656.00000000000...|30114840700000770002|        A|35.00000000000000...|          840|
|      2018-01-01|          2018-01-31|18849846.00000000...|30109810500000435003|        P|34.00000000000000...|          643|
+----------------+--------------------+--------------------+--------------------+---------+--------------------+-------------+
only showing top 3 rows

считывается дельта 1000


AttributeError: 'DataFrame' object has no attribute 'merge'